## Exercise 11b: MPC-based Function Approximation for pointmass task

In this exercise, we look at the problem of function approximation with model predictive control.

### 1. Value Function Approximation and Sensitivity Analysis

In the previous exercises we looked at Neural Networks as function approximators. In this exercise, we will use Model Predictive Control (MPC) to give us a way to approximate the value functions $Q_\theta$, $V_\theta$ and the policy $\pi_\theta$.

We will use your implementation of a parameterized MPC scheme for a linear tracking (PointMass) task. We will then use it to perform sensitivity analysis on the value functions and policy.


## 2D Pointmass System Description

The given Python implementation describes the dynamics of a 2D pointmass system using state-space representation. The state-space matrices `A` and `B` detail the discrete-time dynamics of the pointmass system, widely used to model dynamic systems in control theory.

### State Vector:

The state vector for the 2D pointmass problem is:

$ x = \begin{bmatrix} x_1 \\ x_2 \\ v_1 \\ v_2 \end{bmatrix} $

Where:
- $x_1$ and $x_2$: Positions of the pointmass in the x and y axes, respectively.
- $v_1$ and $v_2$: Velocities of the pointmass in the x and y axes, respectively.

### System Dynamics:

Given the matrices `A` and `B`, the system dynamics are:

$ x_{t+1} = A \cdot x_t + B \cdot u_t $

Where:
- $x_{t+1}$: State vector at the next time step.
- $x_t$: State vector at the current time step.
- $u_t$: Control input at the current time step.

With the provided `A` and `B` matrices, these dynamics convey:

1. The positions $x_1$ and $x_2$ are updated by adding the velocities $v_1$ and $v_2$ multiplied by `dt` (representing the discrete time step).
2. Velocities $v_1$ and $v_2$ undergo dampening (multiplied by 0.9) to simulate resistance or friction.
3. Velocities $v_1$ and $v_2$ are affected by the control input $u_t$, which acts as a force in both directions.

### Control Input:

The control input is represented as:

$ u_t = \begin{bmatrix} u_1 \\ u_2 \end{bmatrix} $

Where:
- $u_1$: Force in the x-direction.
- $u_2$: Force in the y-direction.

### Goal:

The goal is to move the mass into a circle around the origin.



In [1]:
# Experimental setup

import numpy as np
import casadi as csd

import pointmass 

obs_dim, action_dim = 4,2
seed = 1
env_str = "PointMass"
agent_params= {
        "gamma": 0.95,
        "model_str": "pointmass_v0_model",
        "opt_params": {
            "cost_defn": ["fullW", "fullR"],
            "cost_wt": [[3.0, 3.0, 1.0, 1.0], [0.1, 0.1]],
            "horizon": 10
        }
    }

# Environment init
env = getattr(pointmass, env_str)("PointMass", {}, seed)
state, obs = env.reset()
state

array([0.94942468, 0.90516638, 0.        , 0.        ])

### 2. Value Functions

Implement the value function approximation with model predictive control. Use the provided MPC implementation to derive the value functions $V_\theta(s)$, $Q_\theta(s,a)$ and $\pi_\theta(s)$. The file `mpc_formulation_pointmass.py` defines a solver for the underlying NLP.

Quick recap: 
The MPC-based value function approximation is given as

$V_\theta(s) = \min_{x,u,\sigma} J(x,u,\sigma) \\
\qquad \mathrm{s.t.}\, g(x,u,\sigma) = 0 \\
\qquad\quad hu_0(u_0) \leq 0 \\
\qquad\quad hu(u) \leq 0 \\
\qquad\quad hx(x,\sigma) \leq 0 \\
\qquad\quad hs(\sigma)\leq 0$

The MPC-based policy is $\pi_\theta(s) = u_0^\star$

The MPC-based Q function approximation is given as

$Q_\theta(s,a) = \min_{x,u,\sigma} J(x,u,\sigma) \\
\qquad \mathrm{s.t.}\, g(x,u,\sigma) = 0 \\
\qquad\quad u_0 = a \\
\qquad\quad hu(u) \leq 0 \\
\qquad\quad hx(x,\sigma) \leq 0 \\
\qquad\quad hs(\sigma)\leq 0$


a) Implement the functions `def get_V_theta` and `def get_pi_theta` in the following code block.

b) Implement the function `def get_Q_theta` in the following code block. You have to define a separate NLP solver that takes the action $a$ as an additional input. You can use the provided function `define_nlp_solver` as a starting point.

In [2]:
# Casadi NLP formulation

from mpc_formulation import MPCFormulation

class MPCFormulation_ex(MPCFormulation):
    def __init__(self, model, opt_params, gamma) -> None:
        super().__init__(model, opt_params, gamma)
        
        # Define the NLP solver for the optimization problem
        self.define_nlp_solver()

        # Define optimization problem for Q function
        self.define_Q_nlp_solver()
        
    def define_nlp_solver(self):
        """
        Define the NLP solver for the optimization problem.
        NLP formulation:
        
        V(s) = min_{x,u,sigma} J(x,u,sigma)
            s.t. g(x,u,sigma) = 0
                 hu0(u0) <= 0
                 hu(u) <= 0
                 hx(x,sigma) <= 0
                 hs(sigma) <= 0
        """
        # Objective cost
        J = self.objective_cost()

        # Dynamics equality constraints
        g = self.equality_constraints()

        # Inequality constraints
        hu0, hu, hx, hs = self.inequality_constraints()

        # NLP Problem for value function and policy approximation
        G = csd.vertcat(*g)
        Hu = csd.vertcat(*hu0, *hu)
        Hx = csd.vertcat(*hx)
        Hs = csd.vertcat(*hs)
        G_vcsd = csd.vertcat(*g, *hu0, *hu, *hx, *hs)
        lbg = [0] * G.shape[0] + [-np.inf] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        ubg = [0] * G.shape[0] + [0] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        self.lbg_vcsd = csd.vertcat(*lbg)
        self.ubg_vcsd = csd.vertcat(*ubg)

        # NLP Problem for value function and policy approximation
        opts_setting = {
            "ipopt.max_iter": 500,
            "ipopt.print_level": 0,
            "print_time": 0,
            "ipopt.mu_target": self.etau,
            "ipopt.mu_init": self.etau,
            "ipopt.acceptable_tol": 1e-6,
            "ipopt.acceptable_obj_change_tol": 1e-6,
        }
        vnlp_prob = {
            "f": J,
            "x": self.Opt_Vars,
            "p": csd.vertcat(self.Pf, self.P),
            "g": G_vcsd,
        }
        self.nlp_solver = csd.nlpsol("vsolver", "ipopt", vnlp_prob, opts_setting)

    def define_Q_nlp_solver(self):
        """
        Define the NLP solver for the optimization problem.
        NLP formulation:
        
        Q(s,a) = min_{x,u,sigma} J(x,u,sigma)
            s.t. g(x,u,sigma) = 0
                 u0 = a
                 hu(u) <= 0
                 hx(x,sigma) <= 0
                 hs(sigma) <= 0
        """
        # Objective cost
        J = self.objective_cost()

        # Equality constraints
        g = self.equality_constraints()

        # Inequality constraints
        _, hu, hx, hs = self.inequality_constraints()

        # NLP problem for Q function approximation
        # Implement initial action constraint
        u0 = self.U[:, 0]
        action = self.Pf[self.obs_dim : self.obs_dim + self.action_dim]
        g = ...

        G = csd.vertcat(*g)
        Hu = csd.vertcat(*hu)  # No box constraint on u0
        Hx = csd.vertcat(*hx)
        Hs = csd.vertcat(*hs)
        G_qcsd = csd.vertcat(*g, *hu, *hx, *hs)
        lbg = [0] * G.shape[0] + [-np.inf] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        ubg = [0] * G.shape[0] + [0] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        self.lbg_qcsd = csd.vertcat(*lbg)
        self.ubg_qcsd = csd.vertcat(*ubg)

        opts_setting = {
            "ipopt.max_iter": 100,
            "ipopt.print_level": 0,
            "print_time": 0,
            "ipopt.mu_target": self.etau,
            "ipopt.mu_init": self.etau,
            "ipopt.acceptable_tol": 1e-6,
            "ipopt.acceptable_obj_change_tol": 1e-6,
        }
        
        # define the Q-nlp problem and the corresponding solver
        qnlp_prob = ...
        self.qsolver = ...
    

In [3]:
from mpc import MPCfunapprox

class MPCfunapprox_ex(MPCfunapprox, MPCFormulation_ex):
    def __init__(self, agent_params, seed=1):
        # Parameters
        self._parse_agent_params(**agent_params)
        
        # Model init
        self.model = getattr(pointmass, self.model_str)()
        self.obs_dim = self.model.obs_space.shape[0]
        self.action_dim = self.model.action_space.shape[0]

        # Actor initilizaiton (defines self.nlp_solver)
        MPCFormulation_ex.__init__(self, self.model, self.opt_params, self.gamma)

        # Seeded random number generators
        self.rng1 = np.random.default_rng(seed)
        self.rng2 = np.random.default_rng(seed)

        # Initiating parameter values
        self.pf_val = 0.1 * self.rng1.random((self.nPf, 1))
        self.p_val = 0.1 * self.rng1.random((self.nP, 1))
        if "cost_wt" in self.opt_params:
            self.p_val[
                self.iP_cost[0] : self.iP_cost[1]
            ] = self.cost_model.cost_param_init(self.opt_params["cost_wt"])
        self.pf_val[self.iP_cost[1]:], self.p_val[self.iP_cost[1]:] = self.model.model_param_init() 
        
    def act_forward(self, state):
        """
            Evaluate the policy at the current state
        """
        # Forward policy function evaluation for argmin action
        # Setting Solver params
        obs = self.model.get_obs(state)
        pf_val = self.pf_val
        pf_val[: self.obs_dim, :] = obs[:, None]
        p_val = self.p_val
        p = np.concatenate([pf_val, p_val])[:, 0]
        X0 = self.X0

        # run the nlp solver
        soln = ...
        fl = self.nlp_solver.stats()
        if not fl["success"]:
            print("OCP Solver Unsuccessful")

        # u0 as the MPC policy
        act0 = ...
        
        # bookkeeping
        self.X0 = self.update_X0(opt_var[:, 0])
        info = {
            "optimal": fl["success"],
            "soln": soln.copy(),
            "pf": pf_val.copy(),
            "p": p_val.copy(),
        }
        return act0, info
    
    def V_value(self, state):
        """
            Evaluate the value function at the current state
        """
        # Forward policy function evaluation for argmin action
        # Solver params
        obs = self.model.get_obs(state)
        pf_val = self.pf_val
        pf_val[: self.obs_dim, :] = obs[:, None]
        p_val = self.p_val
        X0 = self.model.get_initial_guess(obs, np.zeros(self.action_dim), self.N)

        # run nlp solver
        soln = ...
        fl = self.nlp_solver.stats()
        if not fl["success"]:
            print("OCP Solver Unsuccessful")
        
        # value function from solution
        v_mpc = ...
        
        info = {
            "optimal": fl["success"],
            "soln": soln.copy(),
            "pf": pf_val.copy(),
            "p": p_val.copy(),
        }
        return v_mpc, info

    def Q_value(self, state, action):
        """
            Evaluate the Q function at the current state and action
        """
        # Action-value function evaluation
        obs = self.model.get_obs(state)
        pf_val = self.pf_val
        pf_val[: self.obs_dim, :] = obs[:, None]
        pf_val[self.obs_dim : self.obs_dim + self.action_dim, :] = action[:, None]

        p_val = self.p_val
        X0 = self.model.get_initial_guess(obs, action, self.N)
        X0[: self.action_dim, :] = action[:, None]

        # run Q nlp solver
        qsoln = ...
        fl = self.qsolver.stats()
        if not fl["success"]:
            print("OCP Solver Unsuccessful")

        # Q value from nlp solution
        q_mpc = ...
        
        info = {
            "optimal": fl["success"],
            "soln": qsoln.copy(),
            "pf": pf_val.copy(),
            "p": p_val.copy(),
        }
        return q_mpc, info
    
agent = MPCfunapprox_ex(agent_params, seed)
agent.reset(state)

TypeError: casadi.casadi.vertcat() argument after * must be an iterable, not ellipsis

In [ ]:
# MPC-based policy
state = np.array([0.94942468, 0.90516638, 0.0, 0.0])
act0, info = agent.act_forward(state)
print(f"MPC-based policy action: {act0}")

# Test
act0_true = np.array([-0.99999964, -0.99999962])
print(f"Test result: {int(np.allclose(act0, act0_true, rtol=1e-2, atol=1e-2))}")

In [ ]:
# MPC-based value function
state = np.array([0.94942468, 0.90516638, 0.0, 0.0])
v_mpc, info_v = agent.V_value(state)
print(f"Value function: {v_mpc}")

# Test
v_mpc_true = 105.69474489009102
print(f"Test result: {int(np.allclose(v_mpc, v_mpc_true, rtol=1e-2, atol=1e-2))}")

In [ ]:
# MPC-based Q function
state = np.array([0.94942468, 0.90516638, 0.0, 0.0])
act = np.array([-0.99999964, -0.99999962])
# act = act0 + np.random.normal(scale=0.0, size=(action_dim))

q_mpc, info_q = agent.Q_value(state, act)
print(f"Q function: {q_mpc}")

# Test
q_mpc_true = 105.69474488992536
print(f"Test result: {int(np.allclose(q_mpc, q_mpc_true, rtol=1e-2, atol=1e-2))}")


## Optimal Control Problem for the 2D Pointmass System

Given the dynamics of our 2D pointmass system, we can define an Optimal Control Problem (OCP) to determine the best control inputs over a certain horizon that satisfies our objectives while respecting certain constraints.

### Objective Function:

The objective is typically to minimize (or maximize) a cost function over a certain time horizon $T$. This cost might be a combination of state and control penalties. Formally:

$$ J^* = \min_{u(t)} \int_0^T \ell(x(t), u(t)) dt $$

Where:
- $J^*$ is the optimal cost.
- $\ell(x(t), u(t))$ is the running cost at time $t$, which can be a function of the state $x$ and control $u$.

### Dynamics (Equality Constraints):

The dynamics of our system are represented as equality constraints:

$$ g(x, u) = x_{t+1} - A \cdot x_t - B \cdot u_t = 0 $$

Here, $g(x, u)$ ensures that the state evolution follows the system dynamics. Additional equality constraints can be added to the problem, for example, to ensure that the system starts at a specific state and applies an initial control input.

### State and Control bounds (Inequality Constraints):

State and control bounds are represented as inequality constraints:

$$ h(x, u) \leq 0$$

## Optimal Control Problem:

A general expression for Optimal Control Problems is often be formulated as:


\begin{align*}
\text{Minimize} \quad & \int_0^T \ell(x(t), u(t)) dt \\
\text{Subject to} \quad & g(x, u) = 0 \\
& h(x, u) \leq 0 \\
\end{align*}


Where the objective is to find the control trajectory $u(t)$ over the horizon $T$ that minimizes the cost while satisfying the system dynamics and any additional constraints.

## Lagrange Function for the NLP


The Lagrangian function, often denoted as $\mathcal{L}$, is crucial in optimal control. It's formulated by combining the objective function with the constraints using Lagrange multipliers.

For our defined Optimal Control Problem (OCP), let's denote the Lagrange multipliers as:
- $\lambda(t)$ for the dynamics equality constraints $g(x, u)$
- $\mu(t)$ for the inequality constraints $h(x, u)$

The Lagrange function for our NLP can be expressed as:

$$
\mathcal{L}(x(t), u(t), \lambda(t), \mu(t)) = L(x(t), u(t)) + \lambda(t)^\top g(x, u) + \mu(t)^\top h(x, u)
$$

Where:
- $L(x(t), u(t))$ is the total cost.
- $\lambda(t)^T g(x, u)$ represents the term for the dynamics equality constraint. The transpose indicates that $\lambda$ might be a vector if $g$ represents multiple constraints.
- $\mu_j(t) h_j(x, u)$ accumulates the contribution from all inequality constraints.

The Karush-Kuhn-Tucker (KKT) conditions also introduce the concept of complementary slackness for inequality constraints:

$$
\mu_j(t) h_j(x, u) = 0 \quad \forall j
$$

This ensures that if $h_j(x, u) < 0$ (i.e., the constraint is not active), then $\mu_j(t) = 0$. Conversely, if $h_j(x, u) = 0$ (i.e., the constraint is active), then $\mu_j(t)$ can be non-zero.

To solve the OCP, one would typically derive the necessary conditions from the Lagrangian. This involves calculating derivatives with respect to the state $x(t)$, control $u(t)$, and the Lagrange multipliers. Setting these derivatives to zero provides the conditions for optimality.


## Sensitivities

### Sensitivities of the Value Functions

To use Reinforcement Learning to improve the MPC, we need to provide the sensitivities of the value functions and policy with respect to the paremters $\theta$, i.e. $\nabla_{\theta}V_\theta$, $\nabla_{\theta}Q_\theta$ and $\nabla_{\theta}\pi_\theta$.

You saw in the lecture that the primal-dual solution $z$ of the OCP satisfies the relationship (keep in mind that the Lagrange function for $V_{\theta}$ and $Q_{\theta}$ are in general not equivalent)
$$
    \nabla_{\theta}V_{\theta}(s) = \nabla_{\theta}\mathcal{L}_{\theta}\\
    \nabla_{\theta}Q_{\theta}(s) = \nabla_{\theta}\mathcal{L}_{\theta}.
$$

Use this to implement the sensitivities for $V_\theta$ and $Q_\theta$ below.

### Policy gradient

To compute the gradient of the policy $\nabla_{\theta}\pi_{\theta} = u_0^*$, we need to find $\nabla_{\theta}z_{\theta}$, which is the gradient of the NLP solution $z=(w^\star,\lambda^\star,\mu^\star)$ with respect to the parameters $\theta$.

The NLP solution $z^\star$ satisfies the KKT conditions:
\begin{align*}
	\nabla_{{w}}\mathcal{L}({w}^{\star},{{\lambda}}^{\star},{\mu}^{\star},\theta) &= 0, \\
	g({w}^{\star},\theta) &= 0,\\
	h_i({w}^{\star},\theta){{\mu}}^{\star}_i  &= 0,\\
\end{align*}
with weakly active and inactive inequalities satisfying $h_i({w}^{\star},\theta) \leq 0$, ${{\mu}}^{\star}_i \geq 0$.

We can collect the KKT conditions in a matrix-vector form
$$
r = \begin{bmatrix}\nabla_{w}\mathcal{L}({w}^{\star},\lambda^\star,\mu^\star,\theta) \\ g_i(w) \\ h_i(w)\mu_i\end{bmatrix}
$$
and then compute the sensitivity of the NLP solution with respect to the parameters $\theta$ via the implicit function theorem.

The IFT states that if $r$ is continuously differentiable, then
$$
\frac{\partial r}{\partial z}\frac{\partial z}{\partial \theta} + \frac{\partial r}{\partial \theta} = 0
$$
where $\frac{\partial r}{\partial z}$ is the Jacobian of the KKT conditions with respect to the NLP solution $z$.

Use this observation to implement the policy gradient $\nabla_{\theta}\pi_{\theta}$ below.


In [ ]:
class MPCFormulation_ex_with_sensitivities(MPCFormulation):
    def __init__(self, model, opt_params, gamma) -> None:
        super().__init__(model, opt_params, gamma)
        
        # Define the NLP solver for the optimization problem
        self.define_nlp_solver_with_sensitivities()

        # Define optimization problem for Q function
        self.define_Q_nlp_solver_with_sensitivities()
        
    def define_nlp_solver_with_sensitivities(self):
        """
        Define the NLP solver for the optimization problem.
        NLP formulation:
        
        V(s) = min_{x,u,sigma} J(x,u,sigma)
            s.t. g(x,u,sigma) = 0
                 hu0(u0) <= 0
                 hu(u) <= 0
                 hx(x,sigma) <= 0
                 hs(sigma) <= 0
        """
        # Objective cost
        J = self.objective_cost()

        # Dynamics equality constraints
        g = self.equality_constraints()

        # Inequality constraints
        hu0, hu, hx, hs = self.inequality_constraints()

        # NLP Problem for value function and policy approximation
        G = csd.vertcat(*g)
        Hu = csd.vertcat(*hu0, *hu)
        Hx = csd.vertcat(*hx)
        Hs = csd.vertcat(*hs)
        G_vcsd = csd.vertcat(*g, *hu0, *hu, *hx, *hs)
        lbg = [0] * G.shape[0] + [-np.inf] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        ubg = [0] * G.shape[0] + [0] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        self.lbg_vcsd = csd.vertcat(*lbg)
        self.ubg_vcsd = csd.vertcat(*ubg)

        # NLP Problem for value function and policy approximation
        opts_setting = {
            "ipopt.max_iter": 500,
            "ipopt.print_level": 0,
            "print_time": 0,
            "ipopt.mu_target": self.etau,
            "ipopt.mu_init": self.etau,
            "ipopt.acceptable_tol": 1e-6,
            "ipopt.acceptable_obj_change_tol": 1e-6,
        }
        vnlp_prob = {
            "f": J,
            "x": self.Opt_Vars,
            "p": csd.vertcat(self.Pf, self.P),
            "g": G_vcsd,
        }
        self.nlp_solver = csd.nlpsol("vsolver", "ipopt", vnlp_prob, opts_setting)
        (
            self.Rfun,
            self.dPi,
            self.dLagV,
            self.dRdz,
            self.dRdp,
        ) = self.build_sensitivities(J, G, Hu, Hx, Hs)

    def define_Q_nlp_solver_with_sensitivities(self):
        """
        Define the NLP solver for the optimization problem.
        NLP formulation:
        
        Q(s,a) = min_{x,u,sigma} J(x,u,sigma)
            s.t. g(x,u,sigma) = 0
                 u0 = a
                 hu(u) <= 0
                 hx(x,sigma) <= 0
                 hs(sigma) <= 0
        """
        # Objective cost
        J = self.objective_cost()

        # Equality constraints
        g = self.equality_constraints()

        # Inequality constraints
        _, hu, hx, hs = self.inequality_constraints()

        # NLP problem for Q function approximation
        g.append(self.U[:, 0] - self.Pf[self.obs_dim : self.obs_dim + self.action_dim])

        G = csd.vertcat(*g)
        Hu = csd.vertcat(*hu)
        Hx = csd.vertcat(*hx)
        Hs = csd.vertcat(*hs)
        G_qcsd = csd.vertcat(*g, *hu, *hx, *hs)
        lbg = [0] * G.shape[0] + [-np.inf] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        ubg = [0] * G.shape[0] + [0] * (Hu.shape[0] + Hx.shape[0] + Hs.shape[0])
        self.lbg_qcsd = csd.vertcat(*lbg)
        self.ubg_qcsd = csd.vertcat(*ubg)

        opts_setting = {
            "ipopt.max_iter": 100,
            "ipopt.print_level": 0,
            "print_time": 0,
            "ipopt.mu_target": self.etau,
            "ipopt.mu_init": self.etau,
            "ipopt.acceptable_tol": 1e-6,
            "ipopt.acceptable_obj_change_tol": 1e-6,
        }
        qnlp_prob = {
            "f": J,
            "x": self.Opt_Vars,
            "p": csd.vertcat(self.Pf, self.P),
            "g": G_qcsd,
        }

        self.qsolver = csd.nlpsol("qsolver", "ipopt", qnlp_prob, opts_setting)
        _, _, self.dLagQ, _, _ = self.build_sensitivities(J, G, Hu, Hx, Hs)
        
    def build_sensitivities(self, J, G, Hu, Hx, Hs):
        """
        Computes the sensitivity functions for given cost and constraints in
        J, G, Hu, Hx, Hs

        Parameters
        ----------
        J : Cost function

        G : Equality constraints

        Hu : Input Constraints

        Hx : State Constraints

        Hs : Slack constraints


        Returns
        -------
        Rfun : Casadi function for KKT matrix
        dPi : Casadi function for nable_pi
        dLagfunc : Casadi function for nabla_lagrange
        f_dRdz : Casadi function for derivative of KKT matrix w.r.t. primal-dual variables
        f_dRdp : Casadi function for derivative of KKT matrix w.r.t. parameters

        """
        # Vector of Lagrange multipliers for dynamics equality constraints
        lamb = csd.MX.sym("lambda", G.shape[0])

        # Vector of Lagrange multipliers for input inequality constraints
        mu_u = csd.MX.sym("muu", Hu.shape[0])

        # Vector of Lagrange multipliers for state inequality constraints
        mu_x = csd.MX.sym("mux", Hx.shape[0])

        # Vector of Lagrange multipliers for slack inequality constraints
        mu_s = csd.MX.sym("mus", Hs.shape[0])

        # Vector of Lagrange multipliers
        mult = csd.vertcat(lamb, mu_u, mu_x, mu_s)

        # Lagrange function
        Lag = ...
        Lagfunc = csd.Function("Lag", [self.Opt_Vars, mult, self.Pf, self.P], [Lag])

        # Jacobian of the Lagrangian with respect to the primal decision variables and fixed + learnable parameters
        dLdw = ...
        dLdP = ...
        f_dLdP = csd.Function("dLdP", [self.Opt_Vars, mult, self.Pf, self.P], [dLdP])

        # Build KKT matrix expression
        R_kkt = csd.vertcat(
            ...,
            ...,
            ... + self.etau,
            ... * Hx + self.etau,
            ... * Hs + self.etau,
        )

        # z contains all variables of the lagrangian
        z = csd.vertcat(self.Opt_Vars, lamb, mu_u, mu_x, mu_s)

        dRdz = csd.jacobian(R_kkt, z)

        dRdP = csd.jacobian(R_kkt, self.P)

        # Generate sensitivity of the optimal solution
        dzdP = ...

        Rfun = csd.Function("Rfun", [z, self.Pf, self.P], [R_kkt])

        # Sensitivity function for the policy
        dPi = csd.Function("dPi", [z, self.Pf, self.P], [dzdP])

        # Sensitivity function of the KKT matrix with respect to the solution
        f_dRdz = csd.Function("dRdz", [z, self.Pf, self.P], [dRdz])

        # Sensitivity function of the KKT matrix with respect to the parameters
        f_dRdp = csd.Function("dRdP", [z, self.Pf, self.P], [dRdP])
        
        return Rfun, dPi, f_dLdp, f_dRdz, f_dRdp 

In [ ]:
class MPCfunapprox_ex_with_sensitivities(MPCfunapprox_ex, MPCFormulation_ex_with_sensitivities):
    def __init__(self, agent_params, seed=1):
        # Parameters
        self._parse_agent_params(**agent_params)
        
        # Model init
        self.model = getattr(pointmass, self.model_str)()
        self.obs_dim = self.model.obs_space.shape[0]
        self.action_dim = self.model.action_space.shape[0]

        # Actor initilizaiton (defines self.nlp_solver)
        MPCFormulation_ex_with_sensitivities.__init__(self, self.model, self.opt_params, self.gamma)

        # Seeded random number generators
        self.rng1 = np.random.default_rng(seed)
        self.rng2 = np.random.default_rng(seed)

        # Parameter values
        self.pf_val = 0.1 * self.rng1.random((self.nPf, 1))
        self.p_val = 0.1 * self.rng1.random((self.nP, 1))

        if "cost_wt" in self.opt_params:
            self.p_val[
                self.iP_cost[0] : self.iP_cost[1]
            ] = self.cost_model.cost_param_init(self.opt_params["cost_wt"])
        self.pf_val[self.iP_cost[1]:], self.p_val[self.iP_cost[1]:] = self.model.model_param_init() 

    def dVdP(self, soln, pf_val, p_val, optimal=True):
        """
            Evaluate the gradient of the value function at the current state
        """
        x = soln["x"].full()
        lam_g = soln["lam_g"].full()
        pf_val = pf_val.copy()
        p_val = p_val.copy()
        if optimal:
            dLdP = self.dLagV(x, lam_g, pf_val[:, 0], p_val[:, 0])
            dLdP = dLdP.full()
        else:
            dLdP = np.zeros((1, self.nP))
        return dLdP

    def dQdP(self, soln, pf_val, p_val, optimal=True):
        """
            Evaluate the gradient of the Q function at the current state and action
        """
        x = soln["x"].full()
        lam_g = soln["lam_g"].full()
        pf_val = pf_val.copy()
        p_val = p_val.copy()
        if optimal:
            dLdP = self.dLagQ(x, lam_g, pf_val[:, 0], p_val[:, 0])
            dLdP = dLdP.full()
        else:
            dLdP = np.zeros((1, self.nP))
        return dLdP

    def dPidP(self, soln, pf_val, p_val, optimal=True):
        """
            Evaluate the gradient of the policy at the current state
        """
        x = soln["x"].full()
        lam_g = soln["lam_g"].full()
        z = np.concatenate((x, lam_g), axis=0)
        pf_val = pf_val.copy()
        p_val = p_val.copy()

        if optimal:
            jacob_act = self.dPi(z, pf_val[:, 0], p_val[:, 0]).full()[: self.action_dim, :]
        else:
            jacob_act = np.zeros((self.action_dim, self.nP))
        return jacob_act
    
agent = MPCfunapprox_ex_with_sensitivities(agent_params, seed)
agent.reset(state)

In [ ]:
# Value function sensitivity
state = np.array([0.94942468, 0.90516638, 0.0, 0.0])
v_mpc, info_v = agent.V_value(state)
dVdP = agent.dVdP(info_v["soln"], info_v["pf"], info_v["p"])
print(f"dVdP: {dVdP}")

# Ans: [[36.24543012 34.40993213 -3.72625971 -3.70410381 32.66832904 -3.52757203
#        -3.50679833  1.68430411  1.67329669  1.6626776   1.51018261  1.52035257
#        1.53283252]]

In [ ]:
# Q function sensitivity
state = np.array([0.94942468, 0.90516638, 0.0, 0.0])
act = np.array([-0.99999964, -0.99999962])
# act = act0 + np.random.normal(scale=0.2,size=(action_dim))

q_mpc, info_q = agent.Q_value(state, act)
dQdP = agent.dQdP(info_q["soln"], info_q["pf"], info_q["p"])
print(f"dQdP: {dQdP}")

# Ans: [[36.24542988 34.40993206 -3.72625969 -3.7041037  32.66832911 -3.52757203
#        -3.50679824  1.6843041   1.67329664  1.66267751  1.5101826   1.52035225
#        1.53283191]]

In [ ]:
# Policy sensitivity
state = np.array([0.94942468, 0.90516638, 0.0, 0.0])
act0, info = agent.act_forward(state)
dPidP = agent.dPidP(info["soln"], info["pf"], info["p"])
print(f"dPidP: {dPidP}")

# Ans: [[-2.60575229e-07 -1.23096878e-07 -8.29589384e-08  2.08237986e-08
#  -3.12857508e-18 -9.88710833e-08 -1.12211556e-18  6.62115906e-08
#   3.31094155e-08  9.28482495e-19  6.35532380e-08  3.29424557e-08
#   2.26890985e-19]
# [-2.98102469e-18 -1.48087607e-07 -1.62558009e-18 -1.18212659e-07
#  -2.79828931e-07  2.39185650e-08 -8.87113280e-08  1.22861219e-18
#   3.76293689e-08  7.52669174e-08  1.70542351e-19  3.61171266e-08
#   7.48841692e-08]]

In [ ]:
dPidP_true = np.array([[-2.60575227e-07, -1.23096877e-07, -8.29589381e-08,  2.08237985e-08,
  -1.83864483e-18, -9.88710825e-08,  3.80551797e-19,  6.62115901e-08,
   3.31094152e-08, -4.85389897e-19,  6.35532374e-08,  3.29424554e-08,
  -9.80885961e-20]
 [-6.53866188e-19, -1.48087608e-07,  4.70289191e-20, -1.18212660e-07,
  -2.79828931e-07,  2.39185651e-08, -8.87113281e-08, -8.70489218e-20,
   3.76293690e-08,  7.52669175e-08, -1.42266421e-20,  3.61171266e-08,
   7.48841694e-08]])
print(f"Test result: {int(np.allclose(dPidP, dPidP_true, rtol=1e-2, atol=1e-2))}")